# Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from scipy import stats

# Prepare datasets

In [2]:
df_x = pd.read_csv('../data/dataset_02/data.csv')
df_y = pd.read_csv('../data/dataset_02/answer.csv')

In [3]:
df_x = df_x.drop(labels=['Unnamed: 0'], axis=1)
df_x

In [4]:
df_y = df_y.drop(labels=['Unnamed: 0'], axis=1)
df_y

In [5]:
df = pd.concat([df_x, df_y], axis=1)
df

In [6]:
labels = [str(i) for i in range(672)]

In [7]:
df = df.drop_duplicates(subset=labels, ignore_index=True)
df

In [8]:
df_x = df.drop(labels=['x1', 'y1', 'z1', 'x2', 'y2', 'z2'], axis=1)
df_x

In [9]:
df_y = df.drop(labels=labels, axis=1)
df_y

In [10]:
df_x = np.reshape(df_x.to_numpy(), (-1, 24, 28, 1))
df_x.shape

In [11]:
df_y = df_y.to_numpy()
df_y

In [12]:
train_x, test_x, train_y, test_y = train_test_split(df_x, df_y, train_size=0.9)

train_x, valid_x, train_y, valid_y = train_test_split(train_x,
                                                    train_y,
                                                    test_size=0.33)

In [13]:
print(train_x.shape), print(train_y.shape)
print(valid_x.shape), print(valid_y.shape)
print(test_x.shape), print(test_y.shape)

# CNN

In [14]:
model = keras.Sequential([
    keras.Input(shape=(24, 28, 1)),
    layers.Conv2D(32, (7, 7), activation="relu", data_format='channels_last'),
    layers.MaxPooling2D((2, 2), strides=2),
    layers.Conv2D(64, (3, 3), activation="relu", data_format='channels_last'),
    layers.MaxPooling2D((2, 2), strides=2),
    layers.Flatten(),
    # layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    # layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(6, activation='linear')
])

In [15]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.MeanSquaredError(),
              metrics=[keras.metrics.MeanSquaredError()])

In [16]:
model.summary()

In [20]:
checkpoint_name = 'weights/Weights-{epoch:03d}--{val_loss:.5f}.hdf5'
checkpoint = keras.callbacks.ModelCheckpoint(checkpoint_name,
                                             monitor='val_loss',
                                             verbose=1,
                                             save_best_only=True,
                                             mode='auto')
callbacks_list = [checkpoint]

In [22]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [23]:
history = model.fit(train_x,
                    train_y,
                    batch_size=64,
                    epochs=50,
                    verbose=1,
                    validation_data=(valid_x, valid_y))  # ,
# callbacks=callbacks_list)

In [ ]:
history.history

In [ ]:
sb.scatterplot(
    {'loss': history.history['loss'], 'validation loss': history.history['val_loss']})
plt.ylabel("Значение функции потерь")
plt.xlabel("Номер эпохи")

In [ ]:
sb.scatterplot({'mean_squared_error': history.history['mean_squared_error'],
               'val_mean_squared_error': history.history['val_mean_squared_error']})

In [ ]:
# Load wights file of the best model :
wights_file = '../logs/weights_model_01/Weights-049--17.80290.hdf5'  # choose the best checkpoint
model.load_weights(wights_file)  # load it
model.compile(loss=keras.losses.MeanSquaredError(
), optimizer=keras.optimizers.Adam(), metrics=[keras.metrics.MeanSquaredError()])

In [ ]:
model.summary(expand_nested=True)

In [ ]:
for i, layer in enumerate(model.layers):
    print(i, layer)
    try:
        print("    ", layer.activation)
    except AttributeError:
        print('   no activation attribute')

# Check accuracy

In [ ]:
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
def to_spherical(coor):
    vector = np.array(
        [(coor[3]-coor[0]), (coor[4]-coor[1]), (coor[5]-coor[2])])
    vectorsphere = np.array([np.sqrt((vector**2).sum()), (np.arctan(np.sqrt(
        coor[0]**2 + coor[1]**2)/coor[2])*180/np.pi), (np.arctan(coor[1]/coor[0])*180/np.pi)])
    return vectorsphere

In [ ]:
xtr = model.predict(X_train.iloc[:1])
xtr